In [1]:
import time
import pandas as pd
import numpy as np
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
def visualize_data(df):
    """Visualize the data using various plots."""
    print('\nVisualizing Data...\n')
    start_time = time.time()

    # Example: Distribution of trip durations
    plt.figure(figsize=(10, 6))
    sns.histplot(df['Trip Duration'], bins=20, kde=True)
    plt.title('Distribution of Trip Durations')
    plt.xlabel('Trip Duration (seconds)')
    plt.ylabel('Frequency')
    plt.show()

    # Example: Most Popular Start Stations
    plt.figure(figsize=(12, 6))  # Set the figure size
    sns.countplot(data=df, x='Start Station', order=df['Start Station'].value_counts().index[:10])
    plt.title('Top 10 Most Popular Start Stations')
    plt.xlabel('Start Station')
    plt.ylabel('Number of Trips')
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
    plt.tight_layout()  # Adjust layout for better spacing
    plt.show()
    # Example: show the distribution of user types 
    # Calculate the counts of each user type
    user_type_counts = df['User Type'].value_counts()

    # Create a pie chart
    plt.figure(figsize=(6, 6))
    plt.pie(user_type_counts, labels=user_type_counts.index, autopct='%1.1f%%', startangle=140)
    plt.title('Distribution of User Types')
    plt.show()
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)

In [3]:
def intelligent_features(df):
    """Implement intelligent features based on the data."""
    print('\nImplementing Intelligent Features...\n')
    start_time = time.time()

    # Example: Recommend best times for bike rides based on least busy hours

    # Sort the filtered DataFrame based on 'Start Time' in ascending order
    sorted_df = df.sort_values(by='Start Time')
    filter_df = sorted_df.tail(100).copy()
    # put the hour in column
    filter_df.loc[:, 'hour'] = filter_df['Start Time'].dt.hour
    # Group the data by day of the week and hour, and count the number of trips in each group
    hourly_counts = filter_df.groupby(['day_of_week', 'hour']).size().reset_index(name='Trip Count')

    # Find the least busy hour for each day
    least_busy_hours_per_day = hourly_counts.groupby('day_of_week')['Trip Count'].idxmin()
    least_busy_hours = hourly_counts.loc[least_busy_hours_per_day]

    # Find the overall least busy hour
    overall_least_busy_hour = hourly_counts.loc[hourly_counts['Trip Count'].idxmin()]

    print("Least busy hours for each day:")
    print(least_busy_hours)
    print("\nOverall least busy hour:")
    print(overall_least_busy_hour)

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)

In [4]:
def is_valid_month(month):
    return month.lower() in [month.lower() for month in calendar.month_name[1:]] or month.lower() in "all"


def is_valid_day(day):
    return day.lower() in [day.lower() for day in calendar.day_name] or day.lower() in "all"


In [5]:
def check_data_entry(prompt, valid_entries):
    """
    Function that asks the user to input data and varifies if it's valid
    This simplifies the get_filters() function, where we need to ask the user for three inputs.
    
    Args:
        (str) prompt - message to show the user 
        (list) valid_entries - list of accepted string
    Returens:
        (str) user_input - user's valid input
    
    """
    try:
        user_input = str(input(prompt)).lower()
        while user_input not in valid_entries :
            print('It ooks like your entry is incorrect.')
            user_input = str(input(prompt)).lower()
        
        print("Great! you've chosen:{}\n".format(user_input))
        return user_input
    except:
        print('There seems to be an issue with your input.')


In [22]:
CITY_DATA = {'chicago': 'chicago.csv',
             'new york city': 'new_york_city.csv',
             'washington': 'washington.csv'}

def get_filters():
    """
    Function to ask user for a city, month and day to analyze.
    Returns:
        (str) city - name of the city to analyze 
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by , or "all" to apply no day fillter
    """
    print("Hello! Let's explore some us bikeshare data!")
    valid_cities = CITY_DATA.keys()
    prompt_cities = 'Choose one of the 3 cities (chicago, new york, whashington)'
    city = check_data_entry(prompt_cities, valid_cities)
    
    valid_months = ['all','january','february','march','april','may','june']
    prompt_month= 'choose a month(all,january,february,....,june):'
    month = check_data_entry(prompt_month,valid_months)
    
    valid_days = ['all','monday','tuesday','wednesday','thuresday','friday','saturday']
    prompt_day= 'choose a day(all,monday,tuesday,....,saturday):'
    day = check_data_entry(prompt_day,valid_days)
    
    print('-'*40)
    return city, month, day

In [8]:
def load_data(city, month, day):
    """
    Loads data for the specified city and filters by month and day if applicable.
    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
    Returns:
        df - pandas DataFrame containing city data filtered by month and day
    """

    # load data file into a dataframe
    df = pd.read_csv(CITY_DATA[city])
    print(df.columns)
    # convert the Start Time column to datetime
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    # extract month and day of week from Start Time to create new columns
    df['month'] = df['Start Time'].dt.month
    df['day_of_week'] = df['Start Time'].dt.strftime('%A')

    # filter by month if applicable
    if month != 'all' and month is not None:
        # use the index of the months list to get the corresponding int
        months = ['january', 'february', 'march', 'april', 'may', 'june']
        month_index = months.index(month) + 1

        # filter by month to create the new dataframe
        df = df[df['month'] == month_index]
    # filter by day of week if applicable
    if day != 'all' and day is not None:
        # filter by day of week to create the new dataframe
        df = df[df['day_of_week'] == day.title()]

    return df

In [9]:
def time_stats(df):
    """Displays statistics on the most frequent times of travel."""

    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()
    # display the most common month
    print(f"The most common month is :{calendar.month_name[df['month'].mode().iloc[0]]}")
    # display the most common day of week
    print(f"The most common day is : {df['day_of_week'].mode().iloc[0]}")
    # display the most common start hour
    print(f"The most common hour is :{df['Start Time'].dt.hour.mode().iloc[0]} h")
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)

In [10]:
def station_stats(df):
    """Displays statistics on the most popular stations and trip."""

    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()

    # display most commonly used start station
    print(f"The most common day is : {df['Start Station'].mode().iloc[0]}")
    # display most commonly used end station
    print(f"The most common day is : {df['End Station'].mode().iloc[0]}")
    # display most frequent combination of start station and end station trip
    df["all_Station"] = df['Start Station'].str.cat(df['End Station'], sep='')
    print(f"The most common day is : {df['all_Station'].mode().iloc[0]}")
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)

In [11]:
def trip_duration_stats(df):
    """Displays statistics on the total and average trip duration."""

    print('\nCalculating Trip Duration...\n')
    start_time = time.time()
    # display total travel time
    print(f"The total trip duration equal : {sum(df['Trip Duration'])}")
    # display mean travel time
    print(f"Th avg travel time : {df['Trip Duration'].mean()}")
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)

In [12]:
def user_stats(df, city):
    """Displays statistics on bikeshare users."""

    print('\nCalculating User Stats...\n')
    start_time = time.time()

    # Display counts of user types
    print(f"Number of each user type :{df['User Type'].value_counts()}")
    # Display counts of gender
    if city == "chicago" or city == "new york city":
        print(f"Number of each gender :{df['Gender'].value_counts()}")
    # Display earliest, most recent, and most common year of birth
    if city == "chicago" or city == "new york city":
        print(f"The earliest date of birth is :{df['Birth Year'].min()}")
        print(f"The most recent date of birth is :{df['Birth Year'].max()}")
        print(f"The most common date of birth is :{df['Birth Year'].mode().iloc[0]}")
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-' * 40)



In [14]:

def display_data(df):
    """Display rows after filtering, 5 by 5 to give the user a chance to see the trip data"""
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    i = 0
    while True:
        flag = input(
            "Do you want to see the individual trip data ? answer yes if you want or no if you are not interested"
            "see the data: ").strip().lower()
        if flag == "yes":
            if i < df.shape[0]:
                print(f"Displaying rows [{i + 1}] to [{min(i + 5, df.shape[0])}] of [{df.shape[0]}]")
            else:
                print("There was no more data")
                return False
            # take 5 rows to display
            print(df.iloc[i:i + 5])
            i += 5
        elif flag == "no":
            return False
        else:
            print("Error in input , please enter yes or no")

In [15]:
def main():
    while True:
        city, month, day = get_filters()
        print(city)
        print(month)
        print(day)

        df = load_data(city, month, day)
        visualize_data(df)
        intelligent_features(df)
        time_stats(df)
        station_stats(df)
        trip_duration_stats(df)
        user_stats(df, city)
        print(f"All statics for {city} in {month} and {day} displayed ")
        display_data(df)
        restart = input('\nWould you like to restart? Enter yes or no.\n').strip().lower()
        if restart.lower() != 'yes':
            break

In [23]:
if __name__ == "__main__":
    main()

Hello! Let's explore some us bikeshare data!
Choose one of the 3 cities (chicago, new york, whashington)chcago
It ooks like your entry is incorrect.
Choose one of the 3 cities (chicago, new york, whashington)chicago
Great! you've chosen:chicago

choose a month(all,january,february,....,june):june
Great! you've chosen:june

choose a day(all,monday,tuesday,....,saturday):saturday
Great! you've chosen:saturday

----------------------------------------
chicago
june
saturday


FileNotFoundError: [Errno 2] No such file or directory: 'chicago.csv'